In [1]:
import cv2,os
data_path = 'D:\data'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]

label_dict = dict(zip(categories,labels))
                  
print(label_dict)
print(labels)
print(categories)


{'text detected': 0, 'text not detected': 1}
[0, 1]
['text detected', 'text not detected']


In [2]:
img_size=100
data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path) 
    
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        
        try:
            
            resize = cv2.resize(img,(img_size,img_size))
            data.append(resize)
            target.append(label_dict[category])
            
            
            
        except Exception as e:
            print('Exception',e)

In [ ]:
import numpy as np

data = np.array(data)
target = np.array(target)

from keras.utils import np_utils

new_target = np_utils.to_categorical(target)


Using TensorFlow backend.


In [ ]:
np.save('data',data)
np.save('target',new_target)

In [ ]:
import numpy as np
data = np.load('data.npy')
target = np.load('target.npy')


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv2D(200,(3,3), input_shape = (img_size,img_size,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dense(2))  
model.add(Activation('relu'))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])


In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.2)

In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model', monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

In [ ]:
from keras.models import load_model
import cv2
import numpy as np

In [ ]:
model = load_model('model-001.model')
txt_clsf = cv2.CascadeClassifier('D:\jb\craft.xml')
source = cv2.VideoCapture(0)
label_dict={0:'text  not matched',1:' text matched'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [ ]:
while(True):
    
    ret,img = source.read()
   
    
    img = np.full((512,512,3), 12, np.uint8)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    txts = txt_clsf.detectMultiScale(gray,1.3,5)
    
    for x,y,w,h in aircrafts:
        
        face_img=gray[y:y+w,x:x+w]
        resize = cv2.resize(face_img,(100,100))
        normalize = resized/255.0
        reshape = np.reshape(normalize,(1,100,100,1))
        result = model.predict(reshape)
        
        label = np.argmax(result,axis-1)[0]
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],1)
        cv2.puttext(img,labels_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.0(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1000)
    if(key==27):
        break
    cv2.destroyAllWindows()
    source.release()